In [2]:
import pandas as pd
import numpy as np
import torch
import torchaudio
import textgrids
import torchaudio.transforms as T

SAMPLE_RATE = 16000

In [3]:
#df['audio_files'], df['annotation_files']

class AudioUtils():

    def resample_audio(signal, org_sr):
        resampler = T.Resample(org_sr, SAMPLE_RATE)
        resampled_sig = resampler(signal)
        return resampled_sig
    
    def open(aud_fn):
        sig, sr = torchaudio.load(aud_fn)
        duration = sig.shape[1]/sr
        #print(sig.shape)
        if sr != SAMPLE_RATE:
            sig = AudioUtils.resample_audio(sig, sr)

        # Converting stereo to mono
        if sig.shape[0] == 2:
            sig = sig.mean(dim=0).unsqueeze(0)
        
        if sig.shape[1]%SAMPLE_RATE == 0:
            duration = sig.shape[1]/SAMPLE_RATE
            sig = sig.squeeze().numpy()
        else:
            sig = sig.squeeze().numpy()
            #print(sig_np.shape)
            en = (int)((np.round(duration)+1)*SAMPLE_RATE - sig.shape[0])
            sig = np.pad(sig, (0,en), mode = 'constant')
            #sig = torch.from_numpy(sig_np).unsqueeze(0)
            duration = sig.shape[0] / SAMPLE_RATE
        
        return sig, SAMPLE_RATE, duration
    
    def get_second_wise_mfcc(signal, duration):
        mfcc_list = []
        MFCC = T.MFCC(sample_rate=SAMPLE_RATE, n_mfcc=13)
        for i in range(0,(int)(duration)):
            mfcc_list.append(MFCC(signal[0][i:(i+SAMPLE_RATE)].unsqueeze(0)).squeeze())

        return mfcc_list

In [4]:
class AnnotUtils():

    def get_speech_secs(fname):
        grid = textgrids.TextGrid(fname)
        speech_secs = []
        for i in grid['silences']:
            if i.text == '1':
                #speech_secs.append([(np.round(i.xmin, decimals=2)), (np.round(i.xmax, decimals=2))])
                speech_secs.append([(int)(np.round(i.xmin, decimals=2)), (int)(np.round(i.xmax, decimals=2))])
        return speech_secs
    

    def get_labs_for_secs(speech_secs, duration):
        
        labels = [i*0 for i in range(0, (int)(duration))]

        for i in range(0,len(speech_secs)):

            if speech_secs[i][0] == speech_secs[i][1]:
                labels[speech_secs[i][0]-1] = 1
            else:
                for j in range(speech_secs[i][0], speech_secs[i][1]):
                    
                    labels[j-1] = 1
                    labels[j] = 1

        return labels

In [5]:
df2 = pd.read_csv('secWise_labs.csv')
df2.head()

,audio_list_secs,secWise_labs
0,../audioData/VAD_Kaggle/Data/Audio/Female/PTDB...,0
1,../audioData/VAD_Kaggle/Data/Audio/Female/PTDB...,0
2,../audioData/VAD_Kaggle/Data/Audio/Female/PTDB...,1
3,../audioData/VAD_Kaggle/Data/Audio/Female/PTDB...,1
4,../audioData/VAD_Kaggle/Data/Audio/Female/PTDB...,1


In [7]:
from torch.utils.data import DataLoader, Dataset, random_split

MFCC = T.MFCC(sample_rate=SAMPLE_RATE, n_mfcc=13)

class VAD_Dataset(Dataset):
    def __init__(self, df) -> None:
        #super().__init__()
        self.df = df
        self.sr = 16000

    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):

        aud_part = self.df.loc[index][0]
        labels = self.df.loc[index][1]
        
        if aud_part[-2] == '_':
            n_sec = (int)(aud_part[-1])
            aud_path = aud_part[:-2]+'.wav'
        elif aud_part[-3] == '_':
            n_sec = (int)(aud_part[-2:])
            aud_path = aud_part[:-3]+'.wav'
        #print(n_sec)
        signal00, sam00, dur00 = AudioUtils.open(aud_path)
        sig00 = torch.from_numpy(signal00[n_sec*SAMPLE_RATE:((n_sec+1)*SAMPLE_RATE)]).unsqueeze(0)
        #print(sig00.shape)
        #labels = torch.Tensor(labels).to(dtype=torch.long)
        #mfcc_list = AudioUtils.get_second_wise_mfcc(sig00, dur00)
        mfcc_tens = MFCC(sig00).to(dtype=torch.float)
        #labels = AnnotUtils.get_labs_for_secs(AnnotUtils.get_speech_secs(annot_path), dur)

        return mfcc_tens, labels

/work/dpandya/miniconda3/envs/learning/lib/python3.10/site-packages/torchaudio/functional/functional.py:571: UserWarning: At least one mel filterbank has all zero values. The value for `n_mels` (128) may be set too high. Or, the value for `n_freqs` (201) may be set too low.
  warnings.warn(


In [35]:
myds = VAD_Dataset(df=df2)

In [36]:
num_items = len(myds)
num_train = round(num_items * 0.8)
num_val = num_items - num_train
train_ds, val_ds = random_split(myds, [num_train, num_val])

In [43]:
train_loader = DataLoader(train_ds, batch_size=32, shuffle=True)
test_loader = DataLoader(val_ds, batch_size=32, shuffle=True)

In [32]:
sampl = torch.randn(32,1,13,81)

conv1 = nn.Conv2d(1, 16, kernel_size=(3, 3), padding=1)
relu1 = nn.ReLU()
maxpool1 = nn.MaxPool2d(kernel_size=(2, 2))

conv2 = nn.Conv2d(16,32,kernel_size=(3), padding=1)
relu2 = nn.ReLU()
maxpool2 = nn.MaxPool2d(kernel_size=(2, 2))

x = maxpool1(relu1(conv1(sampl)))
x = maxpool2(relu2(conv2(x)))
x = x.view(x.size(0), -1)
print(x.shape)
'''
conv2 = nn.Conv2d(32, 64, kernel_size=(3, 3), padding=1)
relu2 = nn.ReLU()
maxpool2 = nn.MaxPool2d(kernel_size=(2, 2))
x = maxpool2(relu2(conv2(x)))
x = x.view(x.size(0), -1)  # Flatten the output
print(x.shape)
shp = 64*3*20

#fc1 = nn.Linear(20, 128)
fc1 = nn.Linear(3840, 128)
relu3 = nn.ReLU()
fc2 = nn.Linear(128, 1)
x = fc2(relu3(fc1(x)))
x = torch.sigmoid(x)'''

print(type(torch.round(x)))

torch.Size([32, 1920])
<class 'torch.Tensor'>


In [37]:
import torch.nn as nn

class SpeechDetectionCNN00(nn.Module):
    def __init__(self):
        super(SpeechDetectionCNN00, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=(3, 3), padding=1)
        self.relu1 = nn.ReLU()
        self.maxpool1 = nn.MaxPool2d(kernel_size=(2, 2))

        self.conv2 = nn.Conv2d(16, 32, kernel_size=(3, 3), padding=1)
        self.relu2 = nn.ReLU()
        self.maxpool2 = nn.MaxPool2d(kernel_size=(2, 2))

        self.fc1 = nn.Linear(1920,128)
        self.relu3 = nn.ReLU()
        self.fc2 = nn.Linear(128, 1)  # 2 classes: Speech or No Speech
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.maxpool1(x)

        x = self.conv2(x)
        x = self.relu2(x)
        x = self.maxpool2(x)

        x = x.view(x.size(0), -1)  # Flatten the output
        x = self.fc1(x)
        x = self.relu3(x)
        x = self.fc2(x)
        x = self.sigmoid(x)
        return x

In [38]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

model = SpeechDetectionCNN00().to(device)

criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [39]:
from sklearn.metrics import accuracy_score

epochs = 20

for epoch in range(epochs):

    preds = []
    targets = []
    
    for batch in train_loader:
        
        inputs, labels = batch
        inputs = inputs.to(device, dtype=torch.float)
        labels = labels.to(device).unsqueeze(1).to(dtype=torch.float)

        optimizer.zero_grad()
        outputs = model(inputs)
        #print(outputs.type(), labels.type())
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    
        model.eval()
        with torch.inference_mode():
            targets.extend(labels.cpu().numpy())
            preds.extend(torch.round(outputs).cpu().numpy())
        

    print(f'Epoch [{epoch+1}/{epochs}], Accuracy: {accuracy_score(targets, preds)}, Loss: {loss.item()}')
    

Epoch [1/20], Accuracy: 0.3620910840282232, Loss: 78.5714340209961
Epoch [2/20], Accuracy: 0.3608082103912765, Loss: 57.142860412597656
Epoch [3/20], Accuracy: 0.3608082103912765, Loss: 64.28572082519531
Epoch [4/20], Accuracy: 0.3608082103912765, Loss: 57.142860412597656
Epoch [5/20], Accuracy: 0.3608082103912765, Loss: 71.42857360839844
Epoch [6/20], Accuracy: 0.3608082103912765, Loss: 57.142860412597656
Epoch [7/20], Accuracy: 0.3608082103912765, Loss: 64.28572082519531
Epoch [8/20], Accuracy: 0.3608082103912765, Loss: 71.42857360839844
Epoch [9/20], Accuracy: 0.3608082103912765, Loss: 57.142860412597656
Epoch [10/20], Accuracy: 0.3608082103912765, Loss: 64.28572082519531
Epoch [11/20], Accuracy: 0.3608082103912765, Loss: 57.142860412597656
Epoch [12/20], Accuracy: 0.3608082103912765, Loss: 50.000003814697266
Epoch [13/20], Accuracy: 0.3608082103912765, Loss: 71.42857360839844
Epoch [14/20], Accuracy: 0.3608082103912765, Loss: 50.000003814697266
Epoch [15/20], Accuracy: 0.3608082103

In [44]:
from sklearn.metrics import accuracy_score

preds = []
targets = []

for batch in test_loader:

    inputs, labels = batch
    
    inputs = inputs.to(device, dtype=torch.float)
    labels = labels.to(device).unsqueeze(1).to(dtype=torch.float)
    
    model.eval()
    with torch.inference_mode():
        y_preds = torch.round(model(inputs))

    targets.extend(labels.cpu().numpy())
    preds.extend(y_preds.cpu().numpy())
    

In [45]:
acc = accuracy_score(targets, preds)
print(acc)

0.3576923076923077


In [46]:
from sklearn.metrics import f1_score

f1_score = f1_score(targets, preds)
f1_score

0.0

In [26]:
import torch.nn as nn

samm = np.random.rand(13,81)
samm = torch.from_numpy(samm).to(torch.float)
samm.dtype

torch.float32

In [38]:
m = nn.Conv2d(1, 3, (3), stride=2, padding=(1,1))
ac_layer = nn.ReLU()
norm_la = nn.BatchNorm2d(3)
pool_la = nn.MaxPool2d(4, stride=1)

t_t = ac_layer(m(samm.unsqueeze(0)))
t_t = norm_la(t_t.unsqueeze(0))
t_t = pool_la(t_t)

t_t.shape

torch.Size([1, 3, 4, 38])

In [ ]:
def rando(path):
    if path[-2] == '_':
        n_sec = (int)(path[-1])
        aud_path = path[:-2]+'.wav'
    elif path[-3] == '_':
        n_sec = (int)(path[-2:])
        aud_path = path[:-3]+'.wav'
    signal00, sam00, dur00 = AudioUtils.open(aud_path)
    sig00 = torch.from_numpy(signal00[n_sec*SAMPLE_RATE:((n_sec+1)*SAMPLE_RATE)]).unsqueeze(0)
    return sig00.shape

for i in df2['audio_list_secs']:
    if rando(i) != torch.Size([1, 16000]):
        print('Ehhh', i)
